<a href="https://colab.research.google.com/github/JRopes/CrystalEnergyPredictionWithInvariants/blob/main/Gaussian_Energy_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SETUP**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab_Notebooks/Dissertation/Prediction_Prototyping') 

**IMPORTING LIBRARY DEPENDENCIES**

In [ ]:
import numpy as np
import pickle
import math
import pandas as pd

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RationalQuadratic, RBF, ConstantKernel, Matern, ExpSineSquared

**IMPORTING DATA**

In [ ]:
feature_dir_path = '/content/drive/MyDrive/Colab_Notebooks/Dissertation/Data/AMDs_T2_1000.csv'

In [ ]:
# Importing Data
RAW_DATA = pd.read_csv(feature_dir_path)

# Converting to Numpy Matrix
data = RAW_DATA.to_numpy()

In [ ]:
# Slicing to create feature and label matrices
features = data[:,4:]
labels = data[:,1]

In [ ]:
pickle.dump(features, open("amd_feature_data_gp.p", "wb"))

pickle.dump(labels, open("amd_label_data_gp.p","wb"))

In [ ]:
def data():
  feature_data = pickle.load(open("amd_feature_data_gp.p","rb"))
  label_data = pickle.load(open("amd_label_data_gp.p","rb"))

  # Filling any empty values
  feature_data = np.nan_to_num(feature_data)

  # Only using the first 100 AMD values
  feature_data = feature_data[:,:100]

  ## MinMax Scaler
  feature_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
  X_scaled = feature_scaler.fit_transform(feature_data)
  
  label_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
  y_scaled = label_scaler.fit_transform(label_data.reshape(-1,1))

  # Splitting into training and test data
  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=True)

  return X_train, y_train, X_test, y_test, label_scaler

**ARCHITECTURE**

In [ ]:
X_train, y_train, X_test, y_test, label_scaler = data()

In [ ]:
# Setting kernel
kernel = RationalQuadratic()

In [ ]:
# Setting up Gaussian Process regressor
gpr = GaussianProcessRegressor(kernel=kernel)

**TRAINING**

In [ ]:
# Fitting and optimizing Gaussian Process Regressor on Training data
gpr.fit(X_train, y_train)

GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
                         kernel=RationalQuadratic(alpha=1, length_scale=1),
                         n_restarts_optimizer=0, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=None)

**EVALUATING**

In [ ]:
# Making predictions to get prediction and standard deviation
mean_predictions, std_predictions = gpr.predict(X_test, return_std=True)

In [ ]:
# Inverse Scaling Predictions and Standard deviations
std_predictions = std_predictions.reshape(-1,1)

In [ ]:
scaler = np.divide(std_predictions, mean_predictions)

In [ ]:
mean_predictions = label_scaler.inverse_transform(mean_predictions)
std_predictions = np.multiply(scaler,mean_predictions)
y_test = label_scaler.inverse_transform(y_test)

In [ ]:
# Evaluating Performance
average_loss = 0
average_loss_percentage = 0
average_loss_percentage_rel_range = 0
counter = 0
rms = 0

error_ranges = np.array((0,0,0,0,0,0))

max_value = -999999.99
min_value = 999999.99

for label in y_test:
    if(label > max_value):
        max_value = label
        
    if(label < min_value):
        min_value = label
        
label_range = abs(max_value - min_value)


for i,prediction in enumerate(mean_predictions):
    percentage_difference = abs((abs(prediction - y_test[i]) / y_test[i]) * 100)
    percentage_difference2 = abs((abs(prediction - y_test[i]) / label_range) * 100)
    loss = abs(prediction - y_test[i])
    average_loss += loss

    rms += loss**2

    if(loss <= 1.0):
      error_ranges[0] += 1
    elif(loss <= 2.0):
      error_ranges[1] += 1
    elif(loss <= 4.0):
      error_ranges[2] += 1
    elif(loss <= 8.0):
      error_ranges[3] += 1
    elif(loss <= 10.0):
      error_ranges[4] += 1
    else:
      error_ranges[5] += 1

    average_loss_percentage += percentage_difference
    average_loss_percentage_rel_range += percentage_difference2
    counter += 1

rms = math.sqrt(rms / counter)

print()
print("SUMMARY:")
print()
print("Root Mean Squared Error: " + str(rms))
print("Mean Absolute Error: " + str(average_loss / counter))
print("Mean Absolute Percentage Error: " + str(average_loss_percentage / counter) + "%")
print("Mean Absolute Percentage Error relative to Label Range: " + str(average_loss_percentage_rel_range / counter) + "%")
print("Accuracy: " + str(100 - (average_loss_percentage / counter)) + "%")
print()
print("BREAKDOWN:")
print("   Error <= 1.0 kJ/mol: " + str(error_ranges[0]) + " or " + str((error_ranges[0] / counter) * 100) + "% of Test Set")
print("   Error <= 2.0 kJ/mol: " + str(error_ranges[1]) + " or " + str((error_ranges[1] / counter) * 100) + "% of Test Set")
print("   Error <= 4.0 kJ/mol: " + str(error_ranges[2]) + " or " + str((error_ranges[2] / counter) * 100) + "% of Test Set")
print("   Error <= 8.0 kJ/mol: " + str(error_ranges[3]) + " or " + str((error_ranges[3] / counter) * 100) + "% of Test Set")
print("   Error <= 10.0.0 kJ/mol: " + str(error_ranges[4]) + " or " + str((error_ranges[4] / counter) * 100) + "% of Test Set")
print("   Error > 10.0 kJ/mol: " + str(error_ranges[5]) + " or " + str((error_ranges[5] / counter) * 100) + "% of Test Set")
print("----------------------------------------------------------------------------------------------")


SUMMARY:

Root Mean Squared Error: 6.4027777373918715
Mean Absolute Error: [4.81108546]
Mean Absolute Percentage Error: [3.45023369]%
Mean Absolute Percentage Error relative to Label Range: [5.05987443]%
Accuracy: [96.54976631]%

BREAKDOWN:
   Error <= 1.0 kJ/mol: 190 or 16.725352112676056% of Test Set
   Error <= 2.0 kJ/mol: 159 or 13.996478873239438% of Test Set
   Error <= 4.0 kJ/mol: 251 or 22.095070422535212% of Test Set
   Error <= 8.0 kJ/mol: 315 or 27.72887323943662% of Test Set
   Error <= 10.0.0 kJ/mol: 96 or 8.450704225352112% of Test Set
   Error > 10.0 kJ/mol: 125 or 11.003521126760564% of Test Set
----------------------------------------------------------------------------------------------


**Prediction with Uncertainty**

In [ ]:
instance = 23

In [ ]:
# Showing prediction with 95% confidence interval
print("Prediction: " + str(mean_predictions[instance,0]) + " +- " + str(-1.96 * std_predictions[instance,0]) + " with 95% Confidence || True Label: " + str(y_test[instance]))

Prediction: -129.09632203634513 +- 19.035719491302302 with 95% Confidence || True Label: [-129.6897]


**FILTERING**

In [ ]:
combined_prediction = np.concatenate((mean_predictions,std_predictions),axis=1)

In [ ]:
combined_prediction = np.concatenate((combined_prediction,y_test),axis=1)

In [ ]:
min_boundary = -1000
max_boundary = 1000

label_max = 0
label_min = -1000

In [ ]:
#####
# Setting Parameters for Filtering
#####
percentage_cutoff = 0.15
confidence_factor = -1.96
#-1.96 for 95% OR -1.282 for 80%
#####

In [ ]:
# Calculating the energy boundaries of the predictions and the labels
for instance in combined_prediction:
  prediction = instance[0]
  std = instance[1] * confidence_factor

  if ((prediction + std) > min_boundary):
    min_boundary = prediction + std

  if ((prediction) < max_boundary):
    max_boundary = prediction

  if (instance[2] > label_min):
    label_min = instance[2]

  if (instance[2] < label_max):
    label_max = instance[2]

In [ ]:
# Calculating the energy threshold for the predictions
energy_cutoff_boundary = max_boundary - ((max_boundary - min_boundary) * percentage_cutoff) ##Keep anything below this boundary

In [ ]:
# Calculating the true energy threshold for the labels
label_energy_cutoff = label_max - ((label_max - label_min) * percentage_cutoff) ##We should keep any instances with label between this cutoff and label_max

In [ ]:
# Filtering out predictions above threshold and evaluating choices using labels
counter = 0
correct = 0
kept_instances = 0
tp = 0
positives = 0

for instance in combined_prediction:
  keep = False
  counter += 1

  prediction = instance[0]
  std = instance[1] * confidence_factor

  if (((prediction + std) <= energy_cutoff_boundary) or ((prediction - std) <= energy_cutoff_boundary) or (prediction <= energy_cutoff_boundary)):
    keep = True
    kept_instances += 1

  if (instance[2] <= label_energy_cutoff):
    positives += 1

    if(keep):
      correct += 1
      tp += 1
  
  if (instance[2] > label_energy_cutoff):
    if(not keep):
      correct += 1

In [ ]:
# Results of Filtering
print("Accuracy: " + str(correct/counter))
print("Recall: " + str(tp/positives))
print("Fraction of Original Landscape: " + str(kept_instances/counter))

Accuracy: 0.9639084507042254
